<a href="https://colab.research.google.com/github/StacyChebet/Dyslexicon/blob/master/dyslexicon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dyslexicon CNN Model**

**Dyslexicon is a CNN model that predicts the likelihood that a child is suffering from Dyslexia.**

In [ ]:
#! pip install -q kaggle

In [ ]:
#! mkdir ~/.kaggle

In [ ]:
#! cp kaggle.json ~/.kaggle/

In [ ]:
#! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#! kaggle datasets download drizasazanitaisa/dyslexia-handwriting-dataset

In [3]:
#Importing modules
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import PIL
import PIL.Image
import tensorflow_datasets as tfds
import pathlib
import cv2


In [4]:
!pip install unrar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install tensorflow==2.9.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
! unzip /content/drive/MyDrive/archive.zip

Archive:  /content/drive/MyDrive/archive.zip
  inflating: Dataset Dyslexia_Password WanAsy321.rar  


In [9]:
! unrar x -Y /content/drive/MyDrive/dyslexiaHandwriting.rar

Streaming output truncated to the last 5000 lines.
Extracting  Gambo/Train/Reversal/e-1267.sgNiH.png                         96%  OK 
Extracting  Gambo/Train/Reversal/e-1268.okrK0.png                         96%  OK 
Extracting  Gambo/Train/Reversal/e-1269.WGV1I.png                         96%  OK 
Extracting  Gambo/Train/Reversal/e-1270.AAguI.png                         96%  OK 
Extracting  Gambo/Train/Reversal/e-1271.acDMI.png                         96%  OK 
Extracting  Gambo/Train/Reversal/e-1272.9422e.png                         96%  OK 
Extracting  Gambo/Train/Reversal/e-1273.9WXvR.png                         96%  OK 
Extracting  Gambo/Train/Reversal/e-1274.xyaXZ.png                         96%  OK 
Extracting  Gambo/Train/Reversal/e-1275.Dw004.png                         96%  OK 
Extracting  Gambo/Train/Reversal/e-1276.qiwaE.png                         96%  OK 
Extracting  Gambo/Train/Rever

In [10]:
#Initializing the CNN
classifier = Sequential()

In [11]:
#Step 1 - Convolution
classifier.add(Conv2D(512, (3,3), input_shape = (32,32,3), activation = 'relu'))
classifier.add(Conv2D(512, (3,3), activation = 'relu'))

In [12]:
#Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [13]:
#Adding a second convolutional layer
classifier.add(Conv2D(128, (3,3), activation = 'relu'))
classifier.add(Conv2D(128, (3,3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [14]:
#Step 3 - Flattening
classifier.add(Flatten())

In [15]:
#Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 64, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'softmax'))


In [16]:
#Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [17]:
#Getting the model's summary
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 512)       14336     
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 512)       2359808   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 512)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       589952    
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 128)        0         
 2D)                                                    

In [18]:
#Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)

test_data_gen = ImageDataGenerator(rescale = 1./255)

training_set = train_data_gen.flow_from_directory(r'/content/Gambo/Train',
                                                  target_size = (32,32),
                                                  batch_size = 32,
                                                  class_mode = 'categorical')

Found 151649 images belonging to 3 classes.


In [19]:
test_set = test_data_gen.flow_from_directory(r'/content/Gambo/Test',
                                             target_size = (32,32),
                                             batch_size = 32,
                                             class_mode = 'categorical')

Found 56723 images belonging to 3 classes.


In [ ]:
classifier.fit(training_set,
               steps_per_epoch = 2000,
               epochs = 20,
               validation_data = test_set,
               validation_steps = 2000)

Epoch 1/20
1999/2000 [============================>.] - ETA: 0s - loss: 0.5446 - accuracy: 0.7504

2000/2000 [==============================] - 119s 55ms/step - loss: 0.5445 - accuracy: 0.7505 - val_loss: 0.9450 - val_accuracy: 0.6570
Epoch 2/20
2000/2000 [==============================] - 81s 40ms/step - loss: 0.3459 - accuracy: 0.8323
Epoch 3/20
2000/2000 [==============================] - 81s 41ms/step - loss: 0.2930 - accuracy: 0.8546
Epoch 4/20
2000/2000 [==============================] - 81s 40ms/step - loss: 0.2701 - accuracy: 0.8630
Epoch 5/20
2000/2000 [==============================] - 79s 40ms/step - loss: 0.2564 - accuracy: 0.8680
Epoch 6/20
2000/2000 [==============================] - 80s 40ms/step - loss: 0.2428 - accuracy: 0.8733
Epoch 7/20
2000/2000 [==============================] - 79s 40ms/step - loss: 0.2325 - accuracy: 0.8759
Epoch 8/20
2000/2000 [==============================] - 80s 40ms/step - loss: 0.2251 - accuracy: 0.8802
Epoch 9/20
2000/2000 [==============================] - 80s 40ms/step - loss: 0.2187 - accuracy: 0.8835
Epoch 10/20
2000/2000 [=========

In [ ]:
#Saving the model
#classifier.save('trained_model')

In [ ]:
#Part 3 - Making new predictions
